In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sqlalchemy
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
from sqlalchemy import create_engine
from clickhouse_sqlalchemy import make_session
import clickhouse_connect
import zipfile
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
client = clickhouse_connect.get_client(
    host='clickhouse-02.lenta.tech', 
    port=8443,
    username='a_al_kosheleva',
    password='Gnjrw1HhkByi'
)

In [3]:
def clickhouse_get_df(qry):
    #print(qry)
    engine = create_engine(URL)
    session = make_session(engine)
    exclusives = session.execute(qry)
    exclusives = pd.DataFrame(exclusives.fetchall(), columns=exclusives.keys())
    return exclusives

In [30]:
first_date = pd.to_datetime('2023-07-03')

In [31]:
first_date_final = datetime.strftime(first_date, '%Y-%m-%d')

In [41]:
list_new =[first_date_final]
for i in range(1,7):
    a = first_date + timedelta(i)
    b = datetime.strftime(a, '%Y-%m-%d')
    list_new.append(b)

In [42]:
list_new

['2023-07-03',
 '2023-07-04',
 '2023-07-05',
 '2023-07-06',
 '2023-07-07',
 '2023-07-08',
 '2023-07-09']

In [43]:
first_date = pd.to_datetime('2023-05-29')

In [44]:
first_date_final = datetime.strftime(first_date, '%Y-%m-%d')

In [45]:
list_old =[first_date_final]
for i in range(1,7):
    a = first_date + timedelta(i)
    b = datetime.strftime(a, '%Y-%m-%d')
    list_old.append(b)

In [46]:
list_old

['2023-05-29',
 '2023-05-30',
 '2023-05-31',
 '2023-06-01',
 '2023-06-02',
 '2023-06-03',
 '2023-06-04']

In [73]:
#выгрузка ЛО
final_lo = pd.DataFrame(columns=['dau', 'arpu', 'avg_check',
       'completed_purchase',
       'viewed_catalog',
       'added_to_cart_catalog_dau', 'added_to_cart_catalog_viewed',
       'added_to_cart_catalog_1_viewed','added_to_cart_catalog_2_viewed'])

for i in range(len(list_old)):
    data = list_old[i]
    
    sql = f'''with users AS (
    select distinct user_id, device_id
    from dwh_datamart.daily_aud_aggregate_app
    where event_date = '{data}'
    and any_active > 0 and new_user > 0
    and brand_name = 'LO'),

    events AS (
    select 'event_date' as date,
           count(distinct if(any_active > 0, device_id, null)) as dau,
           count(distinct if(complete_purchase > 0, device_id, null))/count(distinct if(any_active > 0, device_id, null)) as completed_purchase,
           count(distinct if(view_catalog_1 > 0 or view_catalog_2 > 0 or view_catalog_3 > 0, device_id, null))/count(distinct if(any_active > 0, device_id, null)) as viewed_catalog,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(any_active > 0, device_id, null)) as added_to_cart_catalog_dau,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(view_catalog_1 > 0 or view_catalog_2 > 0 or view_catalog_3 > 0, device_id, null)) as added_to_cart_catalog_viewed,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(view_catalog_1 > 0, device_id, null)) as added_to_cart_catalog_1_viewed,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(view_catalog_2 > 0, device_id, null)) as added_to_cart_catalog_2_viewed


    from dwh_datamart.daily_aud_aggregate_app
    where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    and device_id IN (select distinct device_id from users)
    and brand_name = 'LO'
    group by 1),

    orders AS (
    select 'event_date' as date,
           sum(summa)/count(distinct order_id)  as avg_check,
           sum(summa) as all_revenue
    from dwh_datamart.orders
    where created between '{data}' and date_add(day, 7, toDate('{data}'))
    and flag_cancel = 0
    and source IN ('IOS', 'Android')
    and retail_brand_id = '1'
    and buyer_id IN (select distinct user_id from users)
    group by 1)

    select dau, all_revenue/dau as arpu, avg_check,
           completed_purchase,
           viewed_catalog,
           added_to_cart_catalog_dau, added_to_cart_catalog_viewed,
           added_to_cart_catalog_1_viewed,added_to_cart_catalog_2_viewed
    from events e left join orders o on e.date = o.date
    '''
    

    df_stat = client.query_df(sql)
    final_lo = pd.concat([final_lo, df_stat])

In [74]:
final_lo

,dau,arpu,avg_check,completed_purchase,viewed_catalog,added_to_cart_catalog_dau,added_to_cart_catalog_viewed,added_to_cart_catalog_1_viewed,added_to_cart_catalog_2_viewed
0,9113,155.27,2068.77,0.133107,0.356743,0.174366,0.488773,0.512746,0.526333
0,8977,154.02,1995.15,0.126880,0.369945,0.176562,0.477266,0.498114,0.512282
0,9455,164.43,2092.45,0.137916,0.380222,0.184135,0.484284,0.506546,0.522980
0,10168,191.58,2223.73,0.139064,0.386507,0.186467,0.482443,0.503586,0.515778
0,10710,218.54,2226.99,0.154995,0.411298,0.207003,0.503292,0.525231,0.541789
0,11397,212.07,2057.01,0.167851,0.419321,0.219356,0.523122,0.543715,0.559284
0,11463,204.40,2191.86,0.160778,0.428422,0.213469,0.498269,0.519533,0.534046


In [79]:
#выгрузка УТК
final_utk = pd.DataFrame(columns=['dau', 'arpu', 'avg_check',
       'completed_purchase',
       'viewed_catalog',
       'added_to_cart_catalog_dau', 'added_to_cart_catalog_viewed',
       'added_to_cart_catalog_1_viewed','added_to_cart_catalog_2_viewed'])

for i in range(len(list_old)):
    data = list_old[i]
    
    sql = f'''with users AS (
    select distinct user_id, device_id
    from dwh_datamart.daily_aud_aggregate_app
    where event_date = '{data}'
    and any_active > 0 and new_user > 0
    and brand_name = 'Utkonos'),

    events AS (
    select 'event_date' as date,
           count(distinct if(any_active > 0, device_id, null)) as dau,
           count(distinct if(complete_purchase > 0, device_id, null))/count(distinct if(any_active > 0, device_id, null)) as completed_purchase,
           count(distinct if(view_catalog_1 > 0 or view_catalog_2 > 0 or view_catalog_3 > 0, device_id, null))/count(distinct if(any_active > 0, device_id, null)) as viewed_catalog,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(any_active > 0, device_id, null)) as added_to_cart_catalog_dau,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(view_catalog_1 > 0 or view_catalog_2 > 0 or view_catalog_3 > 0, device_id, null)) as added_to_cart_catalog_viewed,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(view_catalog_1 > 0, device_id, null)) as added_to_cart_catalog_1_viewed,
           count(distinct if(add_to_cart_catalog > 0, device_id, null))/count(distinct if(view_catalog_2 > 0, device_id, null)) as added_to_cart_catalog_2_viewed


    from dwh_datamart.daily_aud_aggregate_app
    where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    and device_id IN (select distinct device_id from users)
    and brand_name = 'Utkonos'
    group by 1),

    orders AS (
    select 'event_date' as date,
           sum(summa)/count(distinct order_id)  as avg_check,
           sum(summa) as all_revenue
    from dwh_datamart.orders
    where created between '{data}' and date_add(day, 7, toDate('{data}'))
    and flag_cancel = 0
    and source IN ('IOS', 'Android')
    and retail_brand_id = '2'
    and buyer_id IN (select distinct user_id from users)
    group by 1)

    select dau, all_revenue/dau as arpu, avg_check,
           completed_purchase,
           viewed_catalog,
           added_to_cart_catalog_dau, added_to_cart_catalog_viewed,
           added_to_cart_catalog_1_viewed,added_to_cart_catalog_2_viewed
    from events e left join orders o on e.date = o.date
    '''
    

    df_stat = client.query_df(sql)
    final_utk = pd.concat([final_utk, df_stat])

In [80]:
final_utk

,dau,arpu,avg_check,completed_purchase,viewed_catalog,added_to_cart_catalog_dau,added_to_cart_catalog_viewed,added_to_cart_catalog_1_viewed,added_to_cart_catalog_2_viewed
0,1,0.00,0.00,0.000000,0.000000,0.000000,NaN,NaN,NaN
0,509,143.28,2805.10,0.125737,0.231827,0.125737,0.542373,0.556522,0.581818
0,471,117.50,2305.98,0.140127,0.318471,0.180467,0.566667,0.578231,0.594406
0,590,149.67,2323.92,0.142373,0.279661,0.176271,0.630303,0.645963,0.650000
0,654,85.89,1652.15,0.162080,0.322630,0.197248,0.611374,0.632353,0.664948
0,571,184.59,2108.09,0.180385,0.395797,0.245184,0.619469,0.639269,0.657277
0,485,145.24,2134.59,0.160825,0.334021,0.189691,0.567901,0.589744,0.605263


In [75]:
final_lo.to_excel('final_lo.xlsx')

In [81]:
final_utk.to_excel('final_utk.xlsx')

In [84]:
sql2 = f'''
with users AS (
select distinct user_id, device_id
from dwh_datamart.daily_aud_aggregate_app
where event_date = '{data}'
and any_active > 0 and new_user > 0),

events AS (
select *, 'LO' as brand_name from dwh_datamart.fb_app_events_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
UNION ALL
select *, 'Utkonos' as brand_name from dwh_datamart.fb_app_events_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
),
sessions AS (
select * from dwh_datamart.fb_app_sessions_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
UNION ALL
select * from dwh_datamart.fb_app_sessions_utk where event_date between between '{data}' and date_add(day, 7, toDate('{data}'))
),

et1 AS (
    SELECT
        lower(event_name) AS event_name,
        user_pseudo_id AS device_id,
        e.event_date as event_date,
        brand_name,
        arrayElement(arrayFilter(x -> x.1 =='item_id', event_params).2, 1) as item_id
    from events e
    left join sessions s ON e.session_id = s.id AND e.event_date = s.event_date
    where event_name = 'add_to_cart_catalog'
    and device_id IN (select distinct device_id from users)
    group by 1,2,3,4,5
)

select brand_name, count(item_id)/count(distinct device_id) as items_per_user
from et1
group by 1
order by 1
'''

In [97]:
#выгрузка по товарам новые когорты
items = pd.DataFrame(columns=['brand_name',
       'items_per_user'])

for i in range(len(list_new)):
    data = list_new[i]
    
    sql2 = f'''with users AS (
    select distinct user_id, device_id
    from dwh_datamart.daily_aud_aggregate_app
    where event_date = '{data}'
    and any_active > 0 and new_user > 0),

    events AS (
    select *, 'LO' as brand_name from dwh_datamart.fb_app_events_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    UNION ALL
    select *, 'Utkonos' as brand_name from dwh_datamart.fb_app_events_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    ),
    sessions AS (
    select * from dwh_datamart.fb_app_sessions_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    UNION ALL
    select * from dwh_datamart.fb_app_sessions_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    ),
    et1 AS (
        SELECT
            lower(event_name) AS event_name,
            user_id,
            e.event_date as event_date,
            brand_name,
            arrayElement(arrayFilter(x -> x.1 =='item_id', event_params).2, 1) as item_id
        from events e
        left join sessions s ON e.session_id = s.id AND e.event_date = s.event_date
        where event_name = 'add_to_cart_catalog'
        and user_id IN (select distinct user_id from users)
        group by 1,2,3,4,5
    )

    select brand_name, count(item_id)/count(distinct user_id) as items_per_user
    from et1
    group by 1
    order by 1
    '''
    
    df_stat = client.query_df(sql2)
    items = pd.concat([items, df_stat])

In [99]:
items.to_excel('items.xlsx')

In [100]:
#выгрузка по товарам старые когорты
items = pd.DataFrame(columns=['brand_name',
       'items_per_user'])

for i in range(len(list_old)):
    data = list_old[i]
    
    sql2 = f'''with users AS (
    select distinct user_id, device_id
    from dwh_datamart.daily_aud_aggregate_app
    where event_date = '{data}'
    and any_active > 0 and new_user > 0),

    events AS (
    select *, 'LO' as brand_name from dwh_datamart.fb_app_events_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    UNION ALL
    select *, 'Utkonos' as brand_name from dwh_datamart.fb_app_events_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    ),
    sessions AS (
    select * from dwh_datamart.fb_app_sessions_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    UNION ALL
    select * from dwh_datamart.fb_app_sessions_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    ),
    et1 AS (
        SELECT
            lower(event_name) AS event_name,
            user_id,
            e.event_date as event_date,
            brand_name,
            arrayElement(arrayFilter(x -> x.1 =='item_id', event_params).2, 1) as item_id
        from events e
        left join sessions s ON e.session_id = s.id AND e.event_date = s.event_date
        where event_name = 'add_to_cart_catalog'
        and user_id IN (select distinct user_id from users)
        group by 1,2,3,4,5
    )

    select brand_name, count(item_id)/count(distinct user_id) as items_per_user
    from et1
    group by 1
    order by 1
    '''
    
    df_stat = client.query_df(sql2)
    items = pd.concat([items, df_stat])

In [101]:
items

,brand_name,items_per_user
0,LO,12.224678
0,LO,12.786453
1,Utkonos,15.379310
0,LO,12.942686
1,Utkonos,12.039216
0,LO,12.967958
1,Utkonos,12.745763
0,LO,12.579360
1,Utkonos,11.740260
0,LO,12.274411


In [102]:
items.to_excel('items.xlsx')

In [105]:
#выгрузка по товарам старые когорты
items_total = pd.DataFrame(columns=['items_per_user'])

for i in range(len(list_old)):
    data = list_old[i]
    
    sql2 = f'''with users AS (
    select distinct user_id, device_id
    from dwh_datamart.daily_aud_aggregate_app
    where event_date = '{data}'
    and any_active > 0 and new_user > 0),

    events AS (
    select *, 'LO' as brand_name from dwh_datamart.fb_app_events_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    UNION ALL
    select *, 'Utkonos' as brand_name from dwh_datamart.fb_app_events_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    ),
    sessions AS (
    select * from dwh_datamart.fb_app_sessions_lo where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    UNION ALL
    select * from dwh_datamart.fb_app_sessions_utk where event_date between '{data}' and date_add(day, 7, toDate('{data}'))
    ),
    et1 AS (
        SELECT
            lower(event_name) AS event_name,
            user_id,
            e.event_date as event_date,
            brand_name,
            arrayElement(arrayFilter(x -> x.1 =='item_id', event_params).2, 1) as item_id
        from events e
        left join sessions s ON e.session_id = s.id AND e.event_date = s.event_date
        where event_name = 'add_to_cart_catalog'
        and user_id IN (select distinct user_id from users)
        group by 1,2,3,4,5
    )

    select count(item_id)/count(distinct user_id) as items_per_user
    from et1
    '''
    
    df_stat = client.query_df(sql2)
    items_total = pd.concat([items_total, df_stat])

In [106]:
items_total

,items_per_user
0,12.224678
0,12.855311
0,12.904918
0,12.958601
0,12.539179
0,12.290392
0,12.961990
